In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import pypllon as plon

RGEN = np.random.RandomState(seed=274395387)

In [ ]:
# %load /Users/dsuess/Code/Pythonlibs/cluster_template.ipy
import ipyparallel
from os import environ

CLUSTER_ID = environ.get('CLUSTER_ID', None)
_clients = ipyparallel.Client(cluster_id=CLUSTER_ID)
_view = _clients.load_balanced_view()
print("Kernels available: {}".format(len(_clients)))

In [ ]:
# lower and upper bounds of number of measurements (as a function of size)
LBOUND = lambda dim: 4 * dim - 12
UBOUND = lambda dim: 6 * dim + 20

# Buffer file
OUTFILE = 'noiseless_gauss.h5'

# the maximal dimension to check
MAX_DIM = 10

# Number of random matrices to sample
SAMPLES = 100

# Fraction of measurement vectors sampled from RECR ensemble
# (the rest is sampled from Gaussian ensemble)
INVECS_GENERATOR = plon.invecs_gaussian

# the single error-componenet standard deviation
SIGMA = 0.

# the optimization function to use (from phaselift.routines)
OPTIM_FUNC = plon.lr_recover_l2

In [ ]:
from scipy.linalg import dft
from pypllon.ccg_haar import unitary_haar

def generate_tmats(dim, rgen):
    assert SAMPLES >= 3
    mats = {'ID': np.eye(dim), 'SWAP': np.eye(dim)[::-1]}
    mats.update({'DFT': dft(dim, scale='sqrtn')})
    mats.update({'RAND_%i' % i: unitary_haar(dim, rgen=rgen) 
                 for i in range(SAMPLES - 3)})
    return mats

In [ ]:
import cvxpy
from tools.helpers import get_git_revision_hash

def write_h5_header(h5file):
    rstate = RGEN.get_state()

    h5file.attrs['commit'] = get_git_revision_hash()
    h5file.attrs['random_gen'] = rstate[0]
    h5file.attrs['random_keys'] = rstate[1]
    h5file.attrs['random_pos'] = rstate[2]
    h5file.attrs['random_has_gauss'] = rstate[3]
    h5file.attrs['random_cached_gaussian'] = rstate[4]
    h5file.attrs['cvxpy_version'] = cvxpy.__version__
    
    setupvars = ((key, value) for key, value in globals().items() 
                 if key.isupper())
    for key, value in setupvars:
        try:
            h5file.attrs[key] = value
        except TypeError:
            pass
       
    
def write_h5_invecs(h5group, invecs):
    h5group['INVECS'] = invecs
        
        
def write_h5_intensities(h5group, intensities):
    group = h5group.create_group('INTENSITIES')
    for key, val in intensities.items():
        group[str(key)] = val
       
    
def write_h5_tmats(h5group, tmats):
    group = h5group.create_group('TARGETS')
    for key, val in tmats.items():
        group[str(key)] = val
        
        
def write_h5_result(h5group, nr_measurements, result):
    group = h5group.create_group('RECOV_%i' % nr_measurements)
    group.attrs['NR_MEASUREMENTS'] = nr_measurements
    for name, success, recons, errs in result:
        if not success:
            continue
        group[str(name)] = recons
        group[str(name)].attrs['errs'] = errs

In [ ]:
_clients[:].push({'OPTIM_FUNC': OPTIM_FUNC});

In [ ]:
%%px --local
import pypllon as plon
from cvxpy import SolverError

In [ ]:
import h5py
from tools.helpers import Progress

def recover(args):
    name, invecs, intensities = args
    try:
        recons, err = plon.recover(invecs, intensities, optim_func=OPTIM_FUNC,
                                   reterr=True)
        return name, True, recons, err
    except (SolverError, RuntimeError, ZeroDivisionError) as e:
        print("Failed recovering {} with m={}".format(name, len(invecs)))
        dim = invecs.shape[1]
        return name, False, np.zeros((dim, dim)), np.zeros(dim)


with h5py.File(OUTFILE, 'w') as df:
    write_h5_header(df)
    for dim in range(2, MAX_DIM + 1):
        dimgroup = df.create_group('DIM=%i' % dim)
        dimgroup.attrs['DIM'] = dim

        invecs = INVECS_GENERATOR(dim, UBOUND(dim), rgen=RGEN)
        write_h5_invecs(dimgroup, invecs)

        tmats = generate_tmats(dim, rgen=RGEN)
        write_h5_tmats(dimgroup, tmats)

        intensities = {name: np.abs(invecs @ tmat.T)**2 + SIGMA * RGEN.randn(*invecs.shape)
                       for name, tmat in tmats.items()}
        write_h5_intensities(dimgroup, intensities)

        nr_measurements = np.arange(max(LBOUND(dim), 1), UBOUND(dim) + 1)
        dimgroup.attrs['NR_MEASUREMENTS'] = nr_measurements

        for m in Progress(nr_measurements):
            sequence = [(name, invecs[:m], val[:m]) for name, val in intensities.items()]
            result = _view.map_sync(recover, sequence)
            write_h5_result(dimgroup, m, result)
            df.flush()

In [ ]:
from pypllon.parsers import load_simdata

@np.vectorize
def recons_error(target, recov):
    a, b = plon.fix_phases('rows_by_max', target, recov)
    return np.linalg.norm(a - b)

with h5py.File(OUTFILE, 'r') as infile:
    df = load_simdata(infile)

print("Number of failed reconstructions: {}".
      format(len(df[df.isnull().any(axis=1)])))

df['recons_err'] = recons_error(df['target'], df['recons'])
df['recons_success'] = df['recons_err'] < 1e-2

In [ ]:
p_success = df.groupby(['dim', 'measurements']) \
    .recons_success.mean() \
    .reset_index() \
    .pivot('measurements', 'dim')

x = p_success.columns.levels[1].values
y = p_success.index.values
z = p_success.values
pl.contourf(*np.meshgrid(x, y), z, alpha=0.7)